## Preparation

### Mounting Google Drive

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing Relevant Libraries and moving to location of data file

In [22]:
import torch
%cd /content/drive/MyDrive/2학년 2학기/COSE362-MachineLearning/Hw1

/content/drive/MyDrive/2학년 2학기/COSE362-MachineLearning/Hw1


### Checking GPU available for acceleration

In [23]:
if torch.cuda.is_available():
    print("GPU is enabled and available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not enabled or available. Check the Runtime Type.")

GPU is enabled and available!
Using GPU: NVIDIA A100-SXM4-40GB


## GMM class implementation

In [27]:
from torch.distributions import MultivariateNormal as MVN #importing MultivariateNormal class from torch.distributions.

class GMM:
  #upon initiation of class, the weights/means/covariance matrices of c(number of components) different MultiVariate Normal distributions(MVN) are intialized.
  def __init__(self, data, c):
    self.weights = torch.tensor([1/c for i in range(c)], device="cuda") #initial: even weights
    self.means = data[torch.randint(len(data), (c,))] #initial: random data point chosen as mean for each MVN
    self.cov_matrices = torch.stack([torch.cov(data.T) for _ in range(c)]).to("cuda") #initial: covariance matrix of data points used as initial values for each MVN
    self.data = data
    self.c = c

  #this method accounts for the "Expectation" step of the EM algorithm. It also returns the "old" likelihood later used to check convergence.
  def expectation(self):
    GMMs = MVN(loc = self.means, covariance_matrix = self.cov_matrices) #the MultivariateNormal class supports batch calculations, hence c number of parameters are all passed at the same time.
    log_prob = GMMs.log_prob(self.data.unsqueeze(1)) #log_prob has a shape of (c, t, k) where t is the number of observations, and k is the number of featuers.
    weighted_log_prob = log_prob + torch.log(self.weights) #notice log calculations were used for numeric stability
    weighted_log_sum = torch.logsumexp(weighted_log_prob, dim=1)
    self.responsibility = torch.exp(weighted_log_prob - weighted_log_sum.unsqueeze(1)) #responsibility matrix of shape (t,k) calculated
    return torch.sum(weighted_log_sum) #returning the likelihood mentioned before


  #this method accounts for the "Maximization" step of the EM algorithm. It also returns the "new" likelihood later used to check convergence.
  def maximize(self):
    #calculations for the weights, means, covariance matrices are all in the slides. Notice how all calculations were vectorized in order to lower computational cost
    self.weights = torch.sum(self.responsibility, 0) / len(self.responsibility)
    self.means = torch.div((self.data.T@self.responsibility).T, torch.sum(self.responsibility, 0).unsqueeze(1))
    centered_data = torch.sub(self.data, self.means.unsqueeze(1))
    weighted_centered_data = centered_data * self.responsibility.T.unsqueeze(-1)
    self.cov_matrices = torch.bmm(torch.transpose(centered_data, 1, 2), weighted_centered_data) / torch.sum(self.responsibility, 0).unsqueeze(1).unsqueeze(1)

    #ensuring positive definiteness of covariance matrix by ensuring symmmetry and adding small epsilon value to the diagonal.
    self.cov_matrices += torch.diag_embed(torch.full((self.c, self.cov_matrices.shape[-1]), 1e-3, device="cuda")) #tests revealed that epsilon of 1e-3 does not alter results
    self.cov_matrices = (self.cov_matrices + torch.transpose(self.cov_matrices, 1, 2)) / 2

    #calculating and returning "new" likelihood
    GMMs = MVN(loc = self.means, covariance_matrix = self.cov_matrices)
    log_prob = GMMs.log_prob(self.data.unsqueeze(1))
    weighted_log_prob = log_prob + torch.log(self.weights)
    weighted_log_sum = torch.logsumexp(weighted_log_prob, dim=1)
    return torch.sum(weighted_log_sum)

  #log_likelihood of each observation in input data. Note that in previous methods a single log_likelihood of the entire dataset were returned.
  def log_likelihood(self, input_data):
    GMMs = MVN(loc = self.means, covariance_matrix = self.cov_matrices)
    log_prob = GMMs.log_prob(input_data.unsqueeze(1))
    weighted_log_prob = log_prob + torch.log(self.weights)
    weighted_log_sum = torch.logsumexp(weighted_log_prob, dim=1)
    return weighted_log_sum

  #this method performs the EM algorithm, with maximum iteration of 1000 and a tolerance of 1e-4.
  def EM(self, max_iter=1000, tol=1e-4):
        for i in range(max_iter):
          log_likelihood = self.expectation()
          new_likelihood = self.maximize()
          if i == (max_iter - 1) or abs(new_likelihood - log_likelihood) < tol: #checking convergence
              break

## Using GMM class to perform classification

### Data Preparation

In [32]:
#this is the K value for the K-fold validation. Higher K values would lead to better utilization of limited data.
K = 5

#reading training data(including labels) and converting to a tensor.
train_file_path = "train.txt"
with open(train_file_path, 'r') as f:
  train_data = f.readlines()
data_with_label = []
for data in train_data:
  data_with_label.append([float(x) for x in data.split()])
data_with_label = torch.tensor(data_with_label, dtype=torch.float, device="cuda")

#full dataset divided into class 0/1.
data_class = data_with_label[:, -1]
data_0 = data_with_label[data_class==0]
data_1 = data_with_label[data_class==1]

#computing the size of the K data sections
section_size_0 = len(data_0) // K
section_size_1 = len(data_1) // K

#preparing train data for K fold:
data_sections_0 = torch.zeros([K, section_size_0, 14], device="cuda") #initializing the tensor that will contain each data section(shape: (K, section size, #feature + 1)).
data_sections_1 = torch.zeros([K, section_size_1, 14], device="cuda")
train_sections_0 = torch.zeros([K, (K - 1) * (section_size_0), 13], device="cuda") #intializing the tensor that will contain training data for each K-fold.
train_sections_1 = torch.zeros([K, (K - 1) * (section_size_1), 13], device="cuda")
for k in range(K): #first filling in data section
  data_sections_0[k] = data_0[k * section_size_0 : (k + 1) * section_size_0]
  data_sections_1[k] = data_1[k * section_size_1 : (k + 1) * section_size_1]
for k in range(K): #filling in training data, notice how labels are excluded.
  train_sections_0[k] = torch.cat((data_sections_0[:k, :, :-1], data_sections_0[k + 1:, :, :-1])).flatten(end_dim=1)
  train_sections_1[k] = torch.cat((data_sections_1[:k, :, :-1], data_sections_1[k + 1:, :, :-1])).flatten(end_dim=1)

#preparing test data for K fold:
test_sections = torch.zeros([K, section_size_0 + section_size_1, 14], device="cuda") #initializing tensor that will contain test data for each K-fold.
for k in range(K): #filling in test data
  test_sections[k] = torch.cat((data_sections_0[k], data_sections_1[k]))

### Finding optimal c

**The optimal number of components (c) found can vary due to the stochastic nature of the initialization of the GMM models. Hence the optimal number of components used for the documentation (8) was chosen based on the average error rate for 20 consecutive trials.**

*Due to numeric stability issues(underflows, overflows, etc.), there is an extremely low chance of a singular covariance matrix raising a ValueError for the .fit() method despite the precautions took before. In the case that occurs, simply rerun the code.*

In [37]:
#top bound for # of components to be evaluated
C = 20

errors = [] #array that will contain error data for each # of components
for c in range(2, C + 1):
  error = 0 #used to average the error of each K-fold
  for k in range(K): #start K-fold validation

    #initializing GMM for class 0 and class 1
    GMM_0 = GMM(train_sections_0[k], c)
    GMM_1 = GMM(train_sections_1[k], c)

    #EM algorithm on GMM until convergence
    GMM_0.EM()
    GMM_1.EM()

    #calculating log_likelihood for unlabeled observations in current test data
    GMM_0_likelihood = GMM_0.log_likelihood(test_sections[k, :, :-1])
    GMM_1_likelihood = GMM_1.log_likelihood(test_sections[k, :, :-1])

    #true_class from test data
    true_class = test_sections[k, :, -1]

    #classification by comparing the sizes of the log_likelihood of GMM_0 and GMM_1
    model_likelihood = torch.cat((GMM_0_likelihood.unsqueeze(1), GMM_1_likelihood.unsqueeze(1)), 1)
    model_class = torch.argmax(model_likelihood, dim=1).to("cuda")

    #calculating error by averaging the difference of the true and model class. This works as data can only be classified into 0 or 1.
    error += torch.mean(abs(model_class - true_class))

  #K-fold complete for current component, the error across K-tests are averaged and added to the errors array intialized before.
  print("accuracy for component", c, ":", float(error/K))
  errors.append(float(error/K))

#component analysis complete, optimal_c can now be found
optimal_c = torch.argmin(torch.tensor(errors)) + 2 #since the loop starts from 2, we add it to the index.
print("optimal c for single trial:", int(optimal_c))

accuracy for component 2 : 0.15627072751522064
accuracy for component 3 : 0.15184138715267181
accuracy for component 4 : 0.15084604918956757
accuracy for component 5 : 0.1505972146987915
accuracy for component 6 : 0.14850696921348572
accuracy for component 7 : 0.1479761153459549
accuracy for component 8 : 0.14737890660762787
accuracy for component 9 : 0.14837424457073212
accuracy for component 10 : 0.1478434056043625
accuracy for component 11 : 0.14795951545238495
accuracy for component 12 : 0.14761114120483398
accuracy for component 13 : 0.14759455621242523
accuracy for component 14 : 0.14865627884864807
accuracy for component 15 : 0.1480092853307724
accuracy for component 16 : 0.14812541007995605
accuracy for component 17 : 0.1480424702167511
accuracy for component 18 : 0.1479761153459549
accuracy for component 19 : 0.1482415497303009
accuracy for component 20 : 0.14840742945671082
optimal c for single trial: 8


### Final error calculation for optimal c

In [38]:
#reading test data(including labels) and converting to a tensor.
test_file_path = "test.txt"
with open(test_file_path, 'r') as f:
  test_data = f.readlines()
test_data_l = []
for data in test_data:
  test_data_l.append([float(x) for x in data.split()])
test_data_l = torch.tensor(test_data_l, device="cuda")

#extracting class and data used to calculate log likelihood
test_true_class = test_data_l[:, -1]
test_data_nl = test_data_l[:, :-1]

#using previously prepared data to train the final GMM for class 0 and 1, using optimal c.
GMM_final_0 = GMM(data_0[:, :-1], optimal_c)
GMM_final_1 = GMM(data_1[:, :-1], optimal_c)
GMM_final_0.EM()
GMM_final_1.EM()

#classification and error calculation done similarly as before.
GMM_0_final_likelihood = GMM_final_0.log_likelihood(test_data_nl)
GMM_1_final_likelihood = GMM_final_1.log_likelihood(test_data_nl)
final_model_likelihood = torch.cat((GMM_0_final_likelihood.unsqueeze(1), GMM_1_final_likelihood.unsqueeze(1)), 1)
final_model_class = torch.argmax(final_model_likelihood, dim=1).to("cuda")
final_error = torch.mean(abs(final_model_class - test_true_class))
print("Final error with optimal number of components:", float(final_error))

Final error with optimal number of components: 0.1467820405960083
